# Production of dredging vessels

Trailing Suction Hopper Dredgers (TSHDs) and Water Injection Dredgers (WIDs) are the main vessels used for maintenance dredging in ports and waterways. The efficiency of reallocating/remobilizing sediments by these vessels can be measured by determining the production of these vessels which is the amount of dredged sediments per period of time.

In this notebook, a basic simulation is set up to determine the production of TSHDs and WIDs in a simple network. Some properties such as the location of dredging and discharging are added to the project network along with vessel properties.

The following steps are taken to conduct the simulation:

* Import libraries
* Initialise simpy environment
* Define object classes
* Create objects
  * Create sites
  * Create vessels
  * Create activities
* Register processes and run simpy

### 0. Importing libraries
It starts with importing libraries

In [2]:
#packages related to time, space, and id
import datetime, time

# package(s) related to the simulation (creating the vessel, running the simulation)
import simpy
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot
import openclsim.plugins as plugin

# package(s) needed for inspecting the output
import pandas as pd

# package(s) needed for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# package(s) used for creating and geo-locating the graph (spatial libraries)  
import shapely.geometry
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

### 1. Initialize simpy Environment

In [2]:
# setup environment (simulation time needs to match the project starting time)
simulation_start  = datetime.datetime(2022, 1, 1, 0, 0)
my_env = simpy.Environment(initial_time=simulation_start.timestamp())

### 2. Define Object classes

In [13]:
# create a site object based on desired mixin classes
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource
    ),
    {})


# create a TransportProcessingResource object based on desired mixin classes
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.LoadingFunction,
        core.UnloadingFunction
    ),
    {})

### 3. Create Objects

#### 3.1. Create site objects

In [14]:
# prepare input data for from_site
location_from_site = shapely.geometry.Point(1.1950, 104.1036)
data_from_site = {"env": my_env,
                  "name": "from_site",
                  "geometry": location_from_site,
                  "capacity": 5000000,
                  "level": 4000000
                 }
# instantiate from_site 
from_site = Site(**data_from_site)

# prepare input data for to_site
location_to_site = shapely.geometry.Point(1.2131, 104.1434)
data_to_site = {"env": my_env,
                "name": "to_site",
                "geometry": location_to_site,
                "capacity": 7000000,
                "level": 500000
               }
# instantiate to_site 
to_site = Site(**data_to_site)

#### 3.2. Create vessel object(s)

In [15]:
# prepare input data for sagarmanthan (WID), ham311 (TSHD), and asia(TSHD)
data_sagarmanthan = {"env": my_env,
                     "name": "sagarmanthan",
                     "geometry": location_from_site,
                     "loading_rate": 0,
                     "unloading_rate": 0,
                     "capacity": 0,
                     "compute_v": 0.8}

data_ham311 = {"env": my_env,
                     "name": "ham311",
                     "geometry": location_from_site,
                     "loading_rate": 10,
                     "unloading_rate": 20,
                     "capacity": 3700,
                     "compute_v": lambda x: 11.5}

data_asia = {"env": my_env,
                     "name": "asia",
                     "geometry": location_from_site,
                     "loading_rate": 20,
                     "unloading_rate": 50,
                     "capacity": 11000,
                     "compute_v": lambda x: 16}

# instantiate sagarmanthan
sagarmanthan = TransportProcessingResource(**data_sagarmanthan)
ham311 = TransportProcessingResource(**data_ham311)
asia = TransportProcessingResource(**data_asia)

#### 3.3. Create activity/activities

In [16]:
# initialise registry
registry = {}

In [17]:
# create a list of the sub processes for HAM311
sub_processes = [
    model.BasicActivity(
        env=my_env,
        name="basic activity",
        registry=registry,
        duration=0,
        additional_logs=[ham311]
    ),
    model.BasicActivity(
        env=my_env,
        name="basic activity",
        registry=registry,
        duration=0,
        additional_logs=[asia],
    )]
        
# create a 'sequential activity' that is made up of the 'sub_processes'
sequential_activity = model.SequentialActivity(
    env=my_env,
    name="Sequential activity of basic activities",
    registry=registry,
    sub_processes=sub_processes,
)

### 4. Register processes and run simpy

In [18]:
# initate the simpy processes defined in the 'while activity' and run simpy
model.register_processes([sequential_activity])
my_env.run()

AssertionError: 

### 5. Inspect Results

#### 5.1. Inspect logs
The method plot.get_log_dataframe returns the log of an activity in the form of a dataframe. By adding other activities in a list as the second argument, the Activity can be made more human readable.

In [19]:
display(plot.get_log_dataframe(reporting_activity, [*sub_processes, parallel_activity, reporting_activity]))

NameError: name 'reporting_activity' is not defined

#### 5.2. Visualize Gantt Chart

In [ ]:
plot.get_gantt_chart([while_activity])

In [ ]:
plot.get_gantt_chart([ham311, from_site, to_site],id_map=[while_activity])

In [ ]:
fig = plot.get_step_chart([ham311, from_site, to_site])

### 1. Production of Trailing Suction Hopper Dredger
Production rate of trailing suction hopper dredgers (TSHDs) is a metric for calculating the amount of dredged sediments during a single dredging cycle. The more the production rate of TSHDs, the less time will be spent for conducting the project. The production of TSHDs varies over time based on the sailing distance to discharging area, capacity of hopper, operational hours, and sediment characteristics. 

Different determining factors can affect the production of TSHDs summarized as follows.
* Propulsion power (for both sailing and dredging operations)
* Pumping power (filling the hopper and pumping the sediments ashore)
* Erosion power (jet power installed on both drag head and hopper)
* Vacuum (suction power to elevate the slurry)
* Manpower (efficiency of crew and the level of their proficiency)
* Hopper capacity

The following steps are taken in this notebookn to evaluate the